## IMPORT MODULES


In [2]:
# Modules for web scraping
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
# URL encoding
from requests.utils import requote_uri
# Module for data manipulation
import pandas as pd
# Module for regular expression
import re
# Module for file management
import os
# Module for timing
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



**Geckodriver path**

In [3]:
# Options for Chrome windows
options = webdriver.FirefoxOptions()
options.add_argument("--start-maximized")
options.add_argument('window-size=2560,1440')
options.add_argument('disable-notifications')
options.add_argument('--disable-infobars')
options.add_argument('start-maximized')

# supaya gak muncul mozilla nya
# options.add_argument('--headless')


In [4]:
DRIVER_PATH = 'geckodriver'
driver = webdriver.Chrome(executable_path = DRIVER_PATH,options=options)

---

**URL and Query**

In [6]:
main_url = 'https://shopee.co.id'
# url = 'https://shopee.co.id/search?keyword='
url='https://shopee.co.id/shop/737390/search'
seller = 'headset'
query_encode = requote_uri(seller.strip('').lower())
url_query = url + query_encode
dir_path = os.getcwd()

**Access website**

In [7]:
driver.get(url)
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.shop-search-result-view')))
# driver.implicitly_wait(20)

<selenium.webdriver.remote.webelement.WebElement (session="06333bdd-c90b-4c11-9707-f6424c17c86d", element="e658ef36-8732-4df5-9426-e6a45e0e3725")>

In [8]:
# get scroll heigh
scroll_height = driver.execute_script('return document.documentElement.scrollHeight;')
scroll_height

2447

In [9]:
# Scroll down slowly
scroll_height = driver.execute_script('return document.documentElement.scrollHeight;')
for iter in range(0,scroll_height,200):
    driver.execute_script('window.scrollTo(0, {});'.format(iter))
    time.sleep(1.5)

In [10]:
product_category_search = driver.find_element_by_css_selector('.shopee-shop-collection-filter__body')

In [11]:
product_category=[]
items = product_category_search.find_elements_by_css_selector('div')
for element in items:
    category_name = element.text
    product_category.append(category_name)

product_category
# items

['Semua Produk',
 'Perawatan & Kecantikan',
 'Dapur',
 'Kamar Mandi',
 'Rumah Tangga',
 'Dekorasi']

### COBA SCRAPE PER-PRODUCT

In [18]:
product_list = []

def scrape_detail_product_page(product_link):

    # bag_url = 'https://shopee.co.id/Jims-Honey-Aishiteru-Mini-Bag-Tas-Dompet-Wanita-i.17074378.5836625439'
    driver.get(product_link)

    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "container")))

    scroll_height = driver.execute_script('return document.documentElement.scrollHeight;')
    for iter in range(0,scroll_height,200):
        driver.execute_script('window.scrollTo(0, {});'.format(iter))
        time.sleep(1.5)

    products = driver.find_element_by_css_selector('div.container')

    price_before_disc = "null"
    
    name = products.find_element_by_xpath('//div[@class="attM6y"]').text
    # products.find_element
    if(products.find_element_by_xpath('//div[@class="_2MaBXe"]')):
        price_before_disc = products.find_element_by_xpath('//div[@class="_2MaBXe"]').text
    price_after_dics = products.find_element_by_xpath('//div[@class="Ybrg9j"]').text
    description = products.find_element_by_xpath('//div[@class="_2u0jt9"]/span').text
    stock = products.find_element_by_xpath('//*[@id="main"]/div/div[2]/div[2]/div[2]/div[1]/div[3]/div/div[4]/div/div[3]/div/div[2]/div[2]/div[2]').text
    # weight = products.find_element_by_xpath('')
    variants = products.find_elements_by_xpath('//button[@class="product-variation"]')
#     img_urls = products.find_elements_by_xpath('//div[@class="_2MDwq_"]')
    rating= products.find_element_by_xpath('//div[@class="OitLRu"]').text
    sold_qty = products.find_element_by_xpath('//div[@class="aca9MM"]').text


    product_list.append({
        'name':name,
        'price_before_disc': price_before_disc,
        'price_after_disc': price_after_dics,
        'description':description,
        'variants':[variant.text for variant in variants],
        'url_product': product_link,
        'ratings': rating,
        'sold_qty': sold_qty,
        # 'img_urls':[img_url.find_elements_by_class_name('_2Fw7Qu V1Fpl5').__getattribute__("background-image") for img_url in img_urls]
    })


In [16]:
category_dict = {
    'perawatan_kecantikan':
    'https://shopee.co.id/shop/737390/search?page=0&shopCollection=5915058',
    'dapur':'https://shopee.co.id/shop/737390/search?page=0&shopCollection=5915095',
    'kamar_mandi': 'https://shopee.co.id/shop/737390/search?page=0&shopCollection=5915099',
    'rumah_tangga': 'https://shopee.co.id/shop/737390/search?page=0&shopCollection=5915100',
    'dekorasi':'https://shopee.co.id/shop/737390/search?page=0&shopCollection=5915180',
    
}


### FUNCTION UNTUK KLIK PER ITEM

In [19]:
for x in category_dict:
    driver.get(category_dict[x])
    elements = WebDriverWait(driver, 30).until(EC.visibility_of_all_elements_located((By.XPATH, '//div[@class="shop-search-result-view"]//div/a')))

    scroll_height = driver.execute_script('return document.documentElement.scrollHeight;')
    # scroll_height = driver.execute_script('return document.documentElement.scrollHeight;')

    for iter in range(0,scroll_height,200):
        driver.execute_script('window.scrollTo(0, {});'.format(iter))
        time.sleep(1.5)


    # products = driver.find_elements_by_css_selector('.shop-search-result-view__item')
    for element in elements:
        #get href
        href = element.get_attribute('href')
        print(href)
        #open new window with specific href
        driver.execute_script("window.open('" +href +"');")
        # switch to new window
        driver.switch_to.window(driver.window_handles[1])


        #......now you are on the new window, scrape here
        #example to scrape 'title' in the new window
        scrape_detail_product_page(href)


        #close the new window
        driver.close()
        #back to main window
        driver.switch_to.window(driver.window_handles[0])

https://shopee.co.id/Penahan-Poni-Hair-Velcro-Rambut-Korean-Style-Jepang-salon-aksesoris-X184-i.737390.127500334?position=0


NoSuchElementException: Message: Unable to locate element: //div[@class="_2MaBXe"]


In [37]:
product_list

[{'name': 'Jims Honey - Aishiteru Mini Bag Tas Dompet Wanita',
  'price_before_disc': 'Rp150.000',
  'price_after_disc': 'Rp84.000',
  'description': 'Introducing Aishiteru Mini Sling Bag from Jims Honey. Mini bag yang imut nan lucu dengan pattern stripe diseluruh permukaan tas\nBentuknya yang portrait / vertikal, menjadikannya ramping secara estetika, namun tetap leluasa dalam hal fungsionalitas. Aishiteru bag cocok untuk tempat gadget, buku notes atau pernik kecil lainnya. Ladies, lengkapi koleksi tas Jims Honey mu. This one is another precious catch\n\nTersedia 2 Sekat Besar Muat HP Dan Uang Kertas\nTersedia 1 Resleting Pocket Besar Muat Uang Kertas\nTersedia 4 Slot Kartu\nLengkap Tali Panjang, Bisa Dijadikan Selempang\nFull Bordir Disekeliling Dompet\nMaterial : Kulit Sintestis\nUkuran 18.5 x 11 x 4 cm\nBerat : 200gr\n\nBelanja Lebih Cepat, Mudah Dan Nyaman Di Toko Jims Honey Official ^^',
  'variants': ['Black', 'Navy', 'Grey', 'Red', 'Yellow'],
  'url_product': 'https://shopee.co

In [40]:
df_product = pd.DataFrame.from_dict(product_list)
df_product.to_excel("product_jims_honey.xlsx")